In [1]:
"""
Purpose: To pull down the neuron split suggestions
from the datajoint table and then write them to a spreadsheet

"""

'\nPurpose: To pull down the neuron split suggestions\nfrom the datajoint table and then write them to a spreadsheet\n\n'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from os import sys
sys.path.append("/meshAfterParty/")

In [4]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru
import preprocessing_vp2 as pre

INFO - 2021-01-12 18:55:45,908 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-12 18:55:45,909 - settings - Setting database.user to celiib
INFO - 2021-01-12 18:55:45,910 - settings - Setting database.password to newceliipass
INFO - 2021-01-12 18:55:45,934 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-12 18:55:45,935 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-12 18:55:45,947 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-12 18:55:46,260 - settings - Setting enable_python_native_blobs to True


In [5]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-12 18:55:55,082 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-12 18:55:55,418 - settings - Setting enable_python_native_blobs to True


# Downloading the limb results

In [6]:
minnie.NeuronSplitSuggestions()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,split_results will store the results of how to split the limbs of neuron objects,run_time the amount of time to run (seconds)
864691134947393276,0,0.25,29.00,0,=BLOB=,100.245
864691135012471798,0,0.25,29.00,0,=BLOB=,342.8413
864691135012571126,0,0.25,29.00,0,=BLOB=,217.0619
864691135012905110,0,0.25,29.00,0,=BLOB=,154.318
864691135065032772,0,0.25,29.00,0,=BLOB=,121.376
864691135065033540,0,0.25,29.00,0,=BLOB=,182.2972
864691135081745143,0,0.25,29.00,0,=BLOB=,93.8166
864691135155574372,0,0.25,29.00,0,=BLOB=,230.5511
864691135210051520,0,0.25,29.00,0,=BLOB=,74.0144
864691135274155493,0,0.25,29.00,0,=BLOB=,207.4935


In [7]:
split_suggestions_data = minnie.NeuronSplitSuggestions.fetch(as_dict=True)

# Converting the Data to links

In [11]:
import proofreading_utils as pru

In [85]:
spreadsheet_data = []

import re
def ipython_html_object_to_link(html_obj):
    links = re.findall("href=[\"\'](.*?)[\"\']", html_obj.data)
    return links[0]

for curr_data in split_suggestions_data:
    curr_link = pru.split_info_to_neuroglancer_link(segment_id=curr_data["segment_id"],
                                        split_info = curr_data["split_results"],
                                        output_type="local"
                                       )
    
    curr_link_html = ipython_html_object_to_link(curr_link)

    n_suggested_cuts = len(pru.get_all_coordinate_suggestions(curr_data["split_results"]))
    n_paths_not_cut = pru.get_n_paths_not_cut(curr_data["split_results"])

    print(f"n_suggested_cuts = {n_suggested_cuts}, n_paths_not_cut = {n_paths_not_cut}")

    local_dict = dict(segment_id=curr_data["segment_id"],
                     n_suggested_cuts=n_suggested_cuts,
                     n_paths_not_cut=n_paths_not_cut,
                     link=curr_link_html)
    
    spreadsheet_data.append(local_dict)

import pandas as pd
allen_spreadsheet = pd.DataFrame.from_dict(spreadsheet_data)
allen_spreadsheet.to_csv("allen_spreadsheet.csv", sep=',')

Number of different layers = 3
n_suggested_cuts = 0, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 2, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 6, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 2, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 2, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 4, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 1, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 3, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 1, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 1, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 0, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 2, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 1, n_paths_not_cut = 0
Number of different layers = 3
n_suggested_cuts = 1

# Creating Datajoint Function That Automatically pulls suggestions and generates spreadsheet

In [105]:
du.create_suggested_splits_neuroglancer_spreadsheet(segment_ids=[864691134947393276,864691135012398582],
    output_type="local", #other option is posting to the server
    output_filepath = None,
    output_folder = "./",
    output_filename = "allen_spreadsheet.csv",
    return_dataframe = False,
    )


INFO - 2021-01-12 23:20:11,799 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-12 23:20:11,800 - settings - Setting database.user to celiib
INFO - 2021-01-12 23:20:11,800 - settings - Setting database.password to newceliipass
INFO - 2021-01-12 23:20:11,803 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-12 23:20:12,103 - settings - Setting enable_python_native_blobs to True


In [101]:
segment_ids = minnie.NeuronSplitSuggestions.fetch("segment_id")
segment_ids

array([864691134947393276, 864691135012398582, 864691135012471798,
       864691135012571126, 864691135012905110, 864691135065018692,
       864691135065024068, 864691135065032772, 864691135065033540,
       864691135081745143, 864691135155574372, 864691135210051520,
       864691135274155493, 864691135291777462, 864691135291788214,
       864691135337764326, 864691135337796326, 864691135341171909,
       864691135345134239, 864691135360133191, 864691135373423816,
       864691135373436616, 864691135382996203, 864691135385192277,
       864691135385207893, 864691135394014197, 864691135403709422,
       864691135403794158, 864691135407333321, 864691135428492848,
       864691135440515912, 864691135440585800, 864691135462292893,
       864691135462301853, 864691135467600268, 864691135490263655,
       864691135502075061, 864691135526113627, 864691135526114907,
       864691135538077554, 864691135567752172, 864691135568932230,
       864691135570745325, 864691135570745581, 864691135576925